In [1]:
import os

In [104]:
import datetime
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Band
import numpy as np

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
import math
import scipy.optimize as optim


In [5]:
import pandas as pd

In [6]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [7]:
from bokeh.embed import components


In [8]:
def do_hist(x, bins=10, width = 550, height = 550, title = None):
  hist, edges = np.histogram(x, bins = bins, density=True)
  p_hist = figure(width = width, height = height, title = title)
  p_hist.quad(top = hist, bottom=0, left=edges[:-1], right=edges[1:], alpha = .4)
  return p_hist

In [9]:
X = [random.gauss(mu = 0, sigma = 1) for x in range(100)]
X2 = [x**2 for x in X]
p = do_hist(X2)
show(p)

In [10]:
X =[random.gauss(mu = 0, sigma = 1) for x in range(100)]
X3 = [random.gauss(mu = -1000, sigma = 1) for x in range(100)]
zz = zip(X, X3)
dff = [(x[0] - x[1]) **2  for x in zz]
p  =do_hist(dff)
show(p)


In [21]:
X = [random.gauss(mu = 0, sigma = 1) for x in range(100)]
X2 = [random.gauss(mu = 1, sigma = 1) for x in range(100)]
#X is data, X2 is some type of fit
#calculate SSR from mean
m1 = np.mean(X)
ssr_mean= [(x-m1) **2 for x in X]
ssr_mean
zz = zip(X,X2)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
ssr_fit

def resample_two_samples(sample1, sample2, num_iterations = 100):
    both = sample1 + sample2
    sum1 = []
    sum2 = []
    for i in range(num_iterations):
        random.shuffle(both) 
        new_1 = both[0:len(sample1)] 
        new_2 = both[len(sample1):] 
        sum1.append(np.mean(new_1)) 
        sum2.append(np.mean(new_2)) 
    return sum1, sum2

def combine_resamples(sample1, sample2, resample1, resample2):
    diff1 = [np.mean(sample1 )- x for x in resample1]
    diff2 = [np.mean(sample2)- x for x in resample2]
    both = np.array(diff1) - np.array(diff2)
    return both
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
sum(ssr_mean)
len([x for x in l if x > 1])




0

In [35]:
def diff(x1, x2):
    assert len(x1) == len(x2)
    diffs = []
    for i in range(len(x1)):
        diffs.append((x1[i] - x2[i])**2)
    return diffs
        
    

In [55]:
def test_resample1(initial, ran, ratio):
    x = list(range(1,50))
    y = [exp_func(x, initial =initial, ratio = ratio) + random.randrange(-1 * ran,ran) for x in x]
    y_hat = [exp_func(x, initial =1, ratio = ratio)  for x in x]
    diffs_fit = diff(y, y_hat)
    the_mean = np.mean(y)
    null_ys = [the_mean for x in y]
    diffs_null = diff(y, null_ys )
    res1, res2 = resample_two_samples(diffs_fit, diffs_null)
    ccc = combine_resamples(diffs_fit, diffs_null, res1, res2)
    p_c = [x for x in ccc if x > 0]
    return len(p_c)/len(ccc)
test_resample1(1, 3, 1.3)

0.0

In [61]:
def get_p_value(y, y_hat):
    diffs_fit = diff(y, y_hat)
    the_mean = np.mean(y)
    null_ys = [the_mean for x in y]
    diffs_null = diff(y, null_ys )
    res1, res2 = resample_two_samples(diffs_fit, diffs_null)
    c = combine_resamples(diffs_fit, diffs_null, res1, res2)
    p_c = [x for x in c if x > 0]
    return len(p_c)/len(c)
x = list(range(1,50))
y = [exp_func(x, initial =initial, ratio = ratio) + random.randrange(-1 * 3, 3) for x in x]
y_hat = [exp_func(x, initial =1, ratio = ratio)  for x in x]
get_p_value(y, y_hat)

0.0

In [12]:
def exp_func(x, initial, ratio):
    return initial * np.power(ratio, x - 1)
X = list(range(25))
initial = 1
ratio = 2
X = list(range(1, 25))
y = [exp_func(x, initial, ratio ) + random.gauss(mu = 0, sigma = 1) for x in X]

popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y) )
initial_hat = popt[0]
ratio_hat = popt[1]
ssr_mean =[(x-np.mean(y)) **2 for x in y]
y_fit = [exp_func(x = x, initial= initial_hat, ratio = ratio_hat) for x in X]
zz = zip(y, y_fit)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
len([x for x in l if x > 0])

100

In [13]:

X = list(range(1, 25))
y = [ random.gauss(mu = 10, sigma = 1) for x in X]
popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y) )
initial_hat = popt[0]
ratio_hat = popt[1]
ssr_mean =[(x-np.mean(y)) **1 for x in y]
y_fit = [exp_func(x = x, initial= initial_hat, ratio = ratio_hat) for x in X]
zz = zip(y, y_fit)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
len([x for x in l if x > 0])


0

In [80]:
def make_trend_line_wash():
    df = pd.read_csv('data/seven_day_county.csv')
    df['date'] = pd.to_datetime(df['date'])
    last_date = df['date'].tolist()[-14]
    df_non_king_trend = df[(df['date']>=  last_date)]
    df_king = df[(df['state'] == 'Washington') & (df['county'] == 'King')]
    df_king_trend = df_king[(df_king['date']>=  last_date)]
    nums = df_king_trend['new_cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_king_trend = df_king_trend.assign(X = X)
    labels = df_king_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    print(popt[0], popt[1])
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p_value = get_p_value(nums, y_hat)
    print(p_value)
    ttt = diff(nums, y_hat)
    ppp = np.percentile(ttt, 95)
    ppp_s = math.sqrt(ppp)
    print(ttt, ppp_s)
    y_hat_error = [x + ppp_s for x in y_hat]
    p = figure(x_axis_type = 'datetime', title = 'King {p}'.format(p = round(popt[1],2)), 
                 plot_width = 350 , plot_height = 350, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    p.line(x = labels, y = y_hat_error)
   
    return p
#p.line(x = labels, y = y_fit2)
#show(p)

In [81]:
def make_trend_line_king():
    df = pd.read_csv('data/non_king.csv')
    df['date'] = pd.to_datetime(df['date'])
    last_date = df['date'].tolist()[-14]
    df_non_king_trend = df[(df['date']>=  last_date)]
    nums = df_non_king_trend['cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_non_king_trend = df_non_king_trend.assign(X = X)
    labels = df_non_king_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    print(popt[0], popt[1])
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p_value = get_p_value(nums, y_hat)
    print(p_value)
    p = figure(x_axis_type = 'datetime', title = 'Washington {p}'.format(p = round(popt[1],2)), 
                 plot_width = 350 , plot_height = 350, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    #print(dir(p.axis))
    p.yaxis.axis_label = 'Cases'
    return p
grid = gridplot([make_trend_line_king(), make_trend_line_wash()], ncols = 4)
show(grid)

755.0317538877835 0.9850815117126741
0.51
204.48401312777472 0.9779762731685523
0.29
[894.6720715023274, 110.58597989681451, 255.37679834001676, 760.4466348904671, 126.98765993665648, 1604.5173108899087, 98.7389398572583, 8.455289227749361, 2919.5022454991113, 1360.5664236078308, 113.51683254029201, 499.07564490636514, 2505.5626792857224, 42.648411949719616] 51.48243902012033


In [19]:
script, div = components(grid)
text = """RT rate for King County and Washington State. RT declining (2020-07-30)."""

In [20]:
def output_graphs(script, div, text, 
                 home_page_dir = '/home/henry/projects/covid19/home_page/'):
    with open(os.path.join(home_page_dir, 'script'), 'w') as write_obj:
              write_obj.write(script)
    with open(os.path.join(home_page_dir, 'div'), 'w') as write_obj:
              write_obj.write(div)
    with open(os.path.join(home_page_dir, 'text.txt'), 'w') as write_obj:
              write_obj.write(text)
output_graphs(script, div, text = text)

In [18]:
df_pop = pd.read_csv('data/states_population.csv')
df_pop.head()
df_pop['population_2019'].dtype

dtype('int64')

In [101]:
def error_opt(X, y, y_hat, num_iter = 100):
    fits = []
    inter = []
    for i in range(num_iter):
        d = diff(y, y_hat)
        diffs = []
        for i in range(len(y)):
            diffs.append(y[i] - y_hat[i])
        random.shuffle(diffs)
        y_new = []
        for i in range(len(y)):
            y_new.append(y[i] + diffs[i])
        popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y_new) )
        fits.append(popt[1])
        inter.append(popt[0])
    y_hat_low = [exp_func(initial = np.percentile(inter, 5), ratio = np.percentile(fits, 5), x = x) for x in X]
    y_hat_high = [exp_func(initial = np.percentile(inter, 95), ratio = np.percentile(fits, 95), x = x) for x in X]
    return y_hat_low, y_hat_high

def make_trend_line_wash2():
    df = pd.read_csv('data/seven_day_county.csv')
    df['date'] = pd.to_datetime(df['date'])
    last_date = df['date'].tolist()[-14]
    df_non_king_trend = df[(df['date']>=  last_date)]
    df_king = df[(df['state'] == 'Washington') & (df['county'] == 'King')]
    df_king_trend = df_king[(df_king['date']>=  last_date)]
    nums = df_king_trend['new_cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_king_trend = df_king_trend.assign(X = X)
    labels = df_king_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    print(popt[0], popt[1])
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p_value = get_p_value(nums, y_hat)
    print(p_value)
    y_hat_low, y_hat_high = error_opt(X, nums, y_hat)
    p = figure(x_axis_type = 'datetime', title = 'King {p}'.format(p = round(popt[1],2)), 
                 plot_width = 350 , plot_height = 350, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    p.line(x = labels, y = y_hat_low)
    p.line(x = labels, y = y_hat_high)
    band = Band(base='x', lower='lower', upper='upper', source=source, 
            level='underlay', fill_alpha=1.0, line_width=1, line_color='black')
    return p
show(make_trend_line_wash2())  


204.48401312777472 0.9779762731685523
0.3


In [121]:
def make_trend_line_wash3():
    df = pd.read_csv('data/seven_day_county.csv')
    df['date'] = pd.to_datetime(df['date'])
    last_date = df['date'].tolist()[-14]
    df_non_king_trend = df[(df['date']>=  last_date)]
    df_king = df[(df['state'] == 'Washington') & (df['county'] == 'King')]
    df_king_trend = df_king[(df_king['date']>=  last_date)]
    nums = df_king_trend['new_cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_king_trend = df_king_trend.assign(X = X)
    labels = df_king_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    print(popt[0], popt[1])
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p_value = get_p_value(nums, y_hat)
    print(p_value)
    y_hat_low, y_hat_high = error_opt(X, nums, y_hat)
    p = figure(x_axis_type = 'datetime', title = 'King {p}'.format(p = round(popt[1],2)), 
                 plot_width = 350 , plot_height = 350, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    source = ColumnDataSource({'x':labels, 'upper': y_hat_high, 'lower':y_hat_low})
    band = Band(base='x', lower='lower', upper='upper', source=source, 
            level='underlay', fill_alpha=1.0, line_width=1, line_color='white', fill_color = '#ff9999')
    p.add_layout(band)
    #p.line(x = labels, y = [y_hat[0] for x in labels], line_color = 'black')
    return p
show(make_trend_line_wash3())  

204.48401312777472 0.9779762731685523
0.34


In [145]:
def make_trend_line(df, plot_width = 350, plot_height = 350, title = ''):
    last_date = df['date'].tolist()[-14]
    df_trend = df[(df['date']>=  last_date)]
    nums = df_trend['new_cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_trend = df_trend.assign(X = X)
    labels = df_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p_value = get_p_value(nums, y_hat)
    y_hat_low, y_hat_high = error_opt(X, nums, y_hat)
    p = figure(x_axis_type = 'datetime', title = '{title} {p}'.format(title = title, p = round(popt[1],2)), 
                 plot_width = plot_width , plot_height = plot_height, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    source = ColumnDataSource({'x':labels, 'upper': y_hat_high, 'lower':y_hat_low})
    band = Band(base='x', lower='lower', upper='upper', source=source, 
            level='underlay', fill_alpha=1.0, line_width=1, line_color='white', fill_color = '#ff9999')
    p.add_layout(band)
    return popt[0], popt[1], p_value, p

def do_wash():
    ps = []
    df = pd.read_csv('data/seven_day_county.csv')
    df['date'] = pd.to_datetime(df['date'])
    df_king = df[(df['state'] == 'Washington') & (df['county'] == 'King')]
    intercept, slope, p_value, p = make_trend_line(df_king, title ='King')
    ps.append(p)
    df_state = pd.read_csv('data/states.csv')
    new_cases = df_state['cases']
    df_state =df_state.assign(new_cases = new_cases)
    df_state['date'] = pd.to_datetime(df_state['date'])
    df_wash = df_state[(df_state['state'] == 'Washington')]
    intercept, slope, p_value, p = make_trend_line(df_wash, title= 'Washington')
    ps.append(p)
    df_non_king = pd.read_csv('data/non_king.csv')
    df_non_king['date'] = pd.to_datetime(df_non_king['date'])
    new_cases = df_non_king['cases']
    df_non_king =df_non_king.assign(new_cases = new_cases)
    intercept, slope, p_value, p = make_trend_line(df_non_king, title= 'Non King')
    ps.append(p)
    grid = gridplot(ps, ncols = 3)
    show(grid)
do_wash()


